In [30]:
from keras.models import load_model
import pandas as pd
import numpy as np

In [2]:
model = load_model('best_model.hdf5')

In [8]:
df = pd.read_csv('data/ratings.csv')
movies = pd.read_csv('data/movies.csv')

In [9]:
user_ids = df["userId"].unique().tolist()
movie_ids = df["movieId"].unique().tolist()

user_encoded = {x: i for i, x in enumerate(user_ids)}
user_decoded = {i: x for i, x in enumerate(user_ids)}

movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_decoded = {i: x for i, x in enumerate(movie_ids)}

In [44]:
def recomend(user_id,top_k=10):
    user_watch = df[df.userId == user_id]
    user_no_watch = movies[~movies["movieId"].isin(user_watch.movieId.values)]["movieId"]
    user_no_watch = list(set(user_no_watch).intersection(set(movie_encoded.keys())))
    user_no_watch = [[movie_encoded.get(x)] for x in user_no_watch]
    
    user_encoder = user_encoded.get(user_id)
    
    user_movies = np.hstack(([[user_encoder]] * len(user_no_watch), user_no_watch))
    
    ratings = model.predict([user_movies[:,0],user_movies[:,1]]).flatten()
    
    top_ratings_indices = ratings.argsort()[-top_k:][::-1]
    
    recommended_movie_ids = [movie_decoded.get(user_no_watch[x][0]) for x in top_ratings_indices]

    return user_watch,recommended_movie_ids

In [45]:
def showRecomend(movies_watched,recommended_movie_ids,user_id):
    print("Mostrando recomendaciones para usuario",user_id)
    print("====" * 15)

    print("\nPeliculas con mas altos raitings de usuario")
    print("----" * 15)
    top_movies_user = (
        movies_watched.sort_values(by="rating", ascending=False)
        .head(5)
        .movieId.values
    )
    
    movie_df_rows = movies[movies["movieId"].isin(top_movies_user)]
    for row in movie_df_rows.itertuples():
        print(row.title, ":", row.genres)
        
    
    print("----" * 15)
    print("Top 10 peliculas recomendadas")
    print("----" * 15)
    recommended_movies = movies[movies["movieId"].isin(recommended_movie_ids)]
    for row in recommended_movies.itertuples():
        print(row.title, ":", row.genres)

In [52]:
user_id = df.userId.sample(1).iloc[0]


user_id = 76

user_watched,recommended_movies = recomend(user_id,5)
showRecomend(user_watched,recommended_movies,user_id)

Mostrando recomendaciones para usuario 76

Peliculas con mas altos raitings de usuario
------------------------------------------------------------
Godfather, The (1972) : Crime|Drama
Monty Python and the Holy Grail (1975) : Adventure|Comedy|Fantasy
Goodfellas (1990) : Crime|Drama
Saw (2004) : Horror|Mystery|Thriller
Serenity (2005) : Action|Adventure|Sci-Fi
------------------------------------------------------------
Top 10 peliculas recomendadas
------------------------------------------------------------
Scout, The (1994) : Comedy|Drama
Love and a .45 (1994) : Action|Comedy|Crime
Flash Gordon Conquers the Universe (1940) : Action|Sci-Fi
Wal-Mart: The High Cost of Low Price (2005) : Documentary
Guardians of the Galaxy (2014) : Action|Adventure|Sci-Fi
